In [1]:
# Import required libraries
import os  # for directory and file handling
import cv2  # for image processing
import numpy as np  # for numerical computations
from sklearn.model_selection import train_test_split  # for splitting the dataset
import tensorflow as tf  # for machine learning and neural network

# Explicitly state the image directories
face_images_positive, face_images_negative = 'face_images/positive', 'face_images/negative'

# Define a function for reading, processing, and labeling the images
def get_image_data(images_dir):
    # Initialize empty lists to hold image data and labels
    img_data, img_labels = [], []
    
    # Iterate over all files in the specified directory
    for file in os.listdir(images_dir):
        # Complete path to the image file
        img_filepath = os.path.join(images_dir, file)
        
        # Check if file is an image (ends with .jpg or .png)
        if img_filepath.endswith(('.jpg', '.png')):
            # Read the image file
            img = cv2.imread(img_filepath)
            
            # Convert the color scheme of the image to grayscale
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Resize the image to a uniform size of 64x64 pixels
            img = cv2.resize(img, (64, 64))
            
            # Add the processed image to the list
            img_data.append(img)
            
            # Label the image based on its directory (1 for positive, 0 for negative)
            img_labels.append(1 if images_dir == face_images_positive else 0)
    
    # Return the list of images and their labels
    return img_data, img_labels

# Use the above function to process the images from the two directories
pos_imgs, pos_labels = get_image_data(face_images_positive)
neg_imgs, neg_labels = get_image_data(face_images_negative)

# Concatenate the images and labels from both directories
all_images, all_labels = pos_imgs + neg_imgs, pos_labels + neg_labels

# Split the images and labels into training and testing sets
train_imgs, test_imgs, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

# Convert the lists to numpy arrays and normalize pixel values to between 0 and 1
train_imgs, test_imgs = np.array(train_imgs) / 255.0, np.array(test_imgs) / 255.0

# Add an extra dimension to the image arrays to match the expected input shape for the neural network
train_imgs, test_imgs = np.reshape(train_imgs, (*train_imgs.shape, 1)), np.reshape(test_imgs, (*test_imgs.shape, 1))

# Convert the label lists to numpy arrays
train_labels, test_labels = np.array(train_labels), np.array(test_labels)

# Define the architecture of the neural network
neural_net = tf.keras.models.Sequential()
neural_net.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))  # first convolution layer
neural_net.add(tf.keras.layers.MaxPooling2D((2, 2)))  # first pooling layer
neural_net.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))  # second convolution layer
neural_net.add(tf.keras.layers.MaxPooling2D((2, 2)))  # second pooling layer
neural_net.add(tf.keras.layers.Flatten())  # flatten the input for the dense layer
neural_net.add(tf.keras.layers.Dense(64, activation='relu'))  # dense layer
neural_net.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # output layer

# Set the parameters for the training of the neural network
neural_net.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the neural network on the training set, validating with the testing set
neural_net.fit(train_imgs, train_labels, epochs=10, batch_size=32, validation_data=(test_imgs, test_labels))

# Save the trained neural network model for future use
neural_net.save('face_detection_model.h5')

2023-07-16 16:19:28.940811: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
197/197 [==============================] - 33s 155ms/step - loss: 0.1028 - accuracy: 0.9785 - val_loss: 0.0954 - val_accuracy: 0.9783
Epoch 2/10
197/197 [==============================] - 25s 128ms/step - loss: 0.0678 - accuracy: 0.9821 - val_loss: 0.0996 - val_accuracy: 0.9783
Epoch 3/10
197/197 [==============================] - 25s 125ms/step - loss: 0.0524 - accuracy: 0.9821 - val_loss: 0.0647 - val_accuracy: 0.9783
Epoch 4/10
197/197 [==============================] - 24s 120ms/step - loss: 0.0319 - accuracy: 0.9900 - val_loss: 0.0698 - val_accuracy: 0.9815
Epoch 5/10
197/197 [==============================] - 24s 121ms/step - loss: 0.0235 - accuracy: 0.9935 - val_loss: 0.0574 - val_accuracy: 0.9847
Epoch 6/10
197/197 [==============================] - 24s 121ms/step - loss: 0.0188 - accuracy: 0.9946 - val_loss: 0.1041 - val_accuracy: 0.9796
Epoch 7/10
197/197 [==============================] - 24s 120ms/step - loss: 0.0199 - accuracy: 0.9939 - val_loss: 0.0372 - val_ac